In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np

from scipy.signal import butter, lfilter, freqz
from scipy import signal
data_after_applying_filter = []

cutoff = 3000 #in Hz
fs = 114*1000 # 5micro samples per second
order = 5
NOISE_SIZE = 350
ECHO_SIZE = 200
THRESHOLD = 0.15

def butter_lowpass(cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order):
    b, a = butter_lowpass(cutoff, fs, order=order)
    #y = lfilter(b, a, data)
    y = signal.filtfilt(b, a, data)
    return y

In [98]:
#Step 1 and 2
def get_time_domain_without_offset(filename):
    data_frame = pd.read_csv(filename, skiprows=[0], header= None)
    required_data_frame = data_frame.iloc[:, 9:]
    #print(data_frame)
    #print(required_data)
    required_data_without_offset = required_data_frame.sub(required_data_frame.mean(axis=1), axis=0).values
    return required_data_without_offset

#Step 3

def use_low_pass_filter(data_values):
    data_after_applying_filter = []
    for i, data in enumerate(data_values):
        y = butter_lowpass_filter(data, cutoff, fs, order)
        result = data - y
        data_after_applying_filter.append(result)
    return data_after_applying_filter

# Step 4:
def peak_value(data):
    max_point_distance = 0
    peakData = 0
    max_point_distance = np.array(data).argmax()
    peakData = np.array(data).max()
    if peakData > THRESHOLD:
        return max_point_distance
    else: 
        return None
    
def get_echos(filtered_values):
    all_echo_range = [] 
    for index, data in enumerate(filtered_values):
        chopped_data = data[NOISE_SIZE:]
        max_point_distance = peak_value(chopped_data)
        if max_point_distance:
            cutting_distance = max_point_distance - 100
            if cutting_distance > 0:
                echo_range = chopped_data[cutting_distance:]
                echo_range = echo_range[:ECHO_SIZE]
                all_echo_range.append(echo_range)
    return all_echo_range

def save_to_csv(echo_set, folder, file):
    df = pd.DataFrame(echo_set)
    print(folder,file)
    df.to_csv('./Documents/data_set/Result/{}/{}_overall.csv'.format(folder, file), header=False, index=False)


In [81]:
file_set = [1,2,3,4,5,6,7,8,9]
file = 100
# folder_name = '67_Ford_Black_front'
# folder_name = '26_Ford_Black_back'
# folder_name = '99_Fiesta_Purple_side'
folder_name = 'Pillar'
folder_set = ['67_Ford_Black_front','26_Ford_Black_back','99_Fiesta_Purple_side','Human_A', 'Human_B', 'Pillar']
sub_folder_set = ['Front','Back','Side','Human','Human', 'Pillar']
for i, folder_name in enumerate(folder_set):
    echo_set = []
    sub_folder = sub_folder_set[i]
    print(folder_name,sub_folder)
    for distance in file_set:
        filename = './Documents/data_set/New_Data_Set/{}/{}_{}/{}.csv'.format(folder_name, sub_folder, file,distance)
        time_domain_data_without_offset = get_time_domain_without_offset(filename)
        filtered_values = use_low_pass_filter(time_domain_data_without_offset)
        echos_data =  get_echos(filtered_values)
        echo_set = echo_set + echos_data
    save_to_csv(echo_set, folder_name, file)

67_Ford_Black_front Front
26_Ford_Black_back Back
99_Fiesta_Purple_side Side
Human_A Human
Human_B Human
Pillar Pillar


In [104]:
file_set = [1,2,3,4,5,6]
# distance_set = [70,80,90,110,120,130,140,150,160,170,180,190,200]
distance_set = [70,80,90,100,110]
# folder_name = '67_Ford_Black_front'
# folder_name = '26_Ford_Black_back'
# folder_name = '99_Fiesta_Purple_side'
# folder_set = ['67_Ford_Black_front','26_Ford_Black_back','99_Fiesta_Purple_side','Human_A', 'Human_B', 'Pillar']
folder_set = ['Human_A','Human_B']
sub_folder_set = ['Human','Human']
for i, folder_name in enumerate(folder_set):
    sub_folder = sub_folder_set[i]
    print(folder_name,sub_folder)
    for distance in distance_set:
        echo_set = []
        for file_number in file_set:
            filename = './Documents/data_set/New_Data_Set/{}/{}_{}/{}.csv'.format(folder_name, sub_folder, distance,file_number)
            print(filename)
            time_domain_data_without_offset = get_time_domain_without_offset(filename)
            filtered_values = use_low_pass_filter(time_domain_data_without_offset)
            echos_data =  get_echos(filtered_values)
            echo_set = echo_set + echos_data
        save_to_csv(echo_set, folder_name, distance)

Human_A Human
./Documents/data_set/New_Data_Set/Human_A/Human_70/1.csv
./Documents/data_set/New_Data_Set/Human_A/Human_70/2.csv
./Documents/data_set/New_Data_Set/Human_A/Human_70/3.csv
./Documents/data_set/New_Data_Set/Human_A/Human_70/4.csv
./Documents/data_set/New_Data_Set/Human_A/Human_70/5.csv
./Documents/data_set/New_Data_Set/Human_A/Human_70/6.csv
Human_A 70
./Documents/data_set/New_Data_Set/Human_A/Human_80/1.csv
./Documents/data_set/New_Data_Set/Human_A/Human_80/2.csv
./Documents/data_set/New_Data_Set/Human_A/Human_80/3.csv
./Documents/data_set/New_Data_Set/Human_A/Human_80/4.csv
./Documents/data_set/New_Data_Set/Human_A/Human_80/5.csv
./Documents/data_set/New_Data_Set/Human_A/Human_80/6.csv
Human_A 80
./Documents/data_set/New_Data_Set/Human_A/Human_90/1.csv
./Documents/data_set/New_Data_Set/Human_A/Human_90/2.csv
./Documents/data_set/New_Data_Set/Human_A/Human_90/3.csv
./Documents/data_set/New_Data_Set/Human_A/Human_90/4.csv
./Documents/data_set/New_Data_Set/Human_A/Human_90/5